In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
import pandas as pd
import datetime as datetime
import numpy as np
from dateutil.relativedelta import relativedelta
from sklearn.preprocessing import OneHotEncoder
import matplotlib
import matplotlib.pyplot as plt
#import psycopg2
from scipy.stats import ks_2samp
import scipy.stats as scats
# import visuals as vs
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier


from sklearn import metrics
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif

import yaml

from icu_mortality_prediction import DATA_DIR


In [2]:
from icu_mortality_prediction.src.features import ptnt_demog

# MIMIC-III Critical Care Database

MIMIC-III (Medical Information Mart for Intensive Care III) is a large, freely-available database comprising deidentified health-related data associated with over forty thousand patients who stayed in critical care units of the Beth Israel Deaconess Medical Center between 2001 and 2012.

The database includes information such as demographics, vital sign measurements made at the bedside (~1 data point per hour), laboratory test results, procedures, medications, caregiver notes, imaging reports, and mortality (both in and out of hospital).

MIMIC supports a diverse range of analytic studies spanning epidemiology, clinical decision-rule improvement, and electronic tool development. It is notable for three factors:

it is freely available to researchers worldwide
it encompasses a diverse and very large population of ICU patients
it contains high temporal resolution data including lab results, electronic documentation, and bedside monitor trends and waveforms.

Citations: 
MIMIC-III, a freely accessible critical care database. Johnson AEW, Pollard TJ, Shen L, Lehman L, Feng M, Ghassemi M, Moody B, Szolovits P, Celi LA, and Mark RG. Scientific Data (2016). DOI: 10.1038/sdata.2016.35. Available at: http://www.nature.com/articles/sdata201635

Pollard, T. J. & Johnson, A. E. W. The MIMIC-III Clinical Database http://dx.doi.org/10.13026/C2XW26 (2016).



# IMPORTING DATA
The mimic III database was downloaded and reconstructed locally using posgresql. The database was managed graphically using Portico. 
A query was run on the mimic III database to generate demographic data and data concerning hospital and ICU stays for patients diagnosed with sepsis according to the Angus criteria (Angus et al, 2001. Epidemiology of severe sepsis in the United States; http://www.ncbi.nlm.nih.gov/pubmed/11445675 )

The query was exported from Porticoto the file PTNT_DEMOG_FIRST24.csv. The data was read into a pandas dataframe lab_events.

## IMPORTING DATA

In [3]:
# patient demographic data includes diagnoses and icd9 codes for each patient and each icustay
demographics_filename = 'PTNT_DEMOG_FIRST24.csv'
demographics_filepath = os.path.join(DATA_DIR, 'interim',demographics_filename)
ptnt_demog_data_df = pd.read_csv(demographics_filepath)
# removes duplicate rows

In [4]:
# DEBUG CODE: TAKE ONLY SAMPLE OF DF
ptnt_demog_data_df = ptnt_demog_data_df.head(10000)

In [5]:
ptnt_demog_data2_df = ptnt_demog_data_df[~ptnt_demog_data_df.index.duplicated(keep='first')].copy()
print("imported")

imported


In [6]:
ptnt_demog_data2_df.shape

(10000, 20)

In [7]:
ptnt_demog_data2_df = ptnt_demog.calculate_age_icu_and_hospital_stay_durations(ptnt_demog_data2_df)

Calculating ages, duration of stays


In [8]:
ptnt_demog_data2_df = ptnt_demog.reconfigure_patient_demographics_columns(ptnt_demog_data2_df)

Reconfiguring columns


In [9]:
ptnt_demog_data2_df.columns

Index(['hadm_id', 'age', 'icu_stay', 'hosp_stay', 'icustay_id', 'subject_id',
       'first_careunit', 'gender', 'marital_status', 'ethnicity', 'insurance',
       'admission_type', 'deathtime', 'hospital_expire_flag'],
      dtype='object')

In [10]:
ptnt_demog_data2_df = ptnt_demog.truncate_age_values(ptnt_demog_data2_df)

In [11]:
ptnt_demog_data2_df['age'].describe()

count    9577.000000
mean       66.134384
std        14.986690
min        18.000000
25%        57.000000
50%        70.000000
75%        77.000000
max        89.000000
Name: age, dtype: float64

In [12]:
ptnt_demog_data2_df.columns

Index(['hadm_id', 'age', 'icu_stay', 'hosp_stay', 'icustay_id', 'subject_id',
       'first_careunit', 'gender', 'marital_status', 'ethnicity', 'insurance',
       'admission_type', 'deathtime', 'hospital_expire_flag'],
      dtype='object')

In [13]:
ptnt_demog_data2_df.head()

,hadm_id,age,icu_stay,hosp_stay,icustay_id,subject_id,first_careunit,gender,marital_status,ethnicity,insurance,admission_type,deathtime,hospital_expire_flag
0,145834,76.0,146.0,259.0,211552,3,MICU,M,MARRIED,WHITE,Medicare,EMERGENCY,NaN,0
1,145834,76.0,146.0,259.0,211552,3,MICU,M,MARRIED,WHITE,Medicare,EMERGENCY,NaN,0
2,145834,76.0,146.0,259.0,211552,3,MICU,M,MARRIED,WHITE,Medicare,EMERGENCY,NaN,0
3,145834,76.0,146.0,259.0,211552,3,MICU,M,MARRIED,WHITE,Medicare,EMERGENCY,NaN,0
4,145834,76.0,146.0,259.0,211552,3,MICU,M,MARRIED,WHITE,Medicare,EMERGENCY,NaN,0


In [14]:
ptnt_demog_data_df.head()

,icustay_id,hadm_id,subject_id,first_careunit,gender,marital_status,ethnicity,insurance,admission_type,admittime,dischtime,intime,outtime,deathtime,dob,hospital_expire_flag,icd9_code,icd9_code.1,short_title,seq_num
0,211552,145834,3,MICU,M,MARRIED,WHITE,Medicare,EMERGENCY,2101-10-20 19:08:00,2101-10-31 13:58:00,2101-10-20 19:10:11,2101-10-26 20:43:09,NaN,2025-04-11 00:00:00,0,0389,0389,Septicemia NOS,1
1,211552,145834,3,MICU,M,MARRIED,WHITE,Medicare,EMERGENCY,2101-10-20 19:08:00,2101-10-31 13:58:00,2101-10-20 19:10:11,2101-10-26 20:43:09,NaN,2025-04-11 00:00:00,0,5849,5849,Acute kidney failure NOS,3
2,211552,145834,3,MICU,M,MARRIED,WHITE,Medicare,EMERGENCY,2101-10-20 19:08:00,2101-10-31 13:58:00,2101-10-20 19:10:11,2101-10-26 20:43:09,NaN,2025-04-11 00:00:00,0,4275,4275,Cardiac arrest,4
3,211552,145834,3,MICU,M,MARRIED,WHITE,Medicare,EMERGENCY,2101-10-20 19:08:00,2101-10-31 13:58:00,2101-10-20 19:10:11,2101-10-26 20:43:09,NaN,2025-04-11 00:00:00,0,41071,41071,"Subendo infarct, initial",5
4,211552,145834,3,MICU,M,MARRIED,WHITE,Medicare,EMERGENCY,2101-10-20 19:08:00,2101-10-31 13:58:00,2101-10-20 19:10:11,2101-10-26 20:43:09,NaN,2025-04-11 00:00:00,0,4280,4280,CHF NOS,6


In [15]:
diagnoses_bm, diagnoses = ptnt_demog.create_diagnoses_defs(ptnt_demog_data_df)

# #phenotypes = add_hcup_ccs_2015_groups(diagnoses, yaml.load(open(args.phenotype_definitions, 'r')))
# print("creating diagnoses definitions")
# definitions_path = os.path.join(DATA_DIR, 'external/hcup_ccs_2015_definitions.yaml')
# definitions = yaml.load(open(definitions_path, 'r'), Loader=yaml.FullLoader)

# diagnoses = ptnt_demog_data_df[['hadm_id', 'icd9_code', 'short_title']].copy()

# # create mapping of hcup_ccs_2015_definitions to diagnoses icd9 codes
# def_map = {}
# for dx in definitions:
#     for code in definitions[dx]['codes']:
#         def_map[code] = (dx, definitions[dx]['use_in_benchmark'])

        

creating diagnoses definitions
creating diagnoses definitions
map created


In [ ]:
print("Calling continuous to categorical conversion")
continuous_to_categorical(ptnt_demog_data)
#print(ptnt_demog_data.head())
print("Calling categorical to dummy variables")
dummies = categorical_to_dummies(ptnt_demog_data)
dummies = dummies.merge(diagnoses2, left_index = True, right_index = True, 
                           how = 'left')
print(dummies.head())

write_best_features(dummies)
print("Patient demographic pre-processing and feature selection complete")

In [ ]:
for col in ptnt_demog_data2.columns: 
    print(col)

In [ ]:

print("patient demographics with unique icustays")


# create patient demographic table with unique icustays as rows

dates_and_times = ['dob', 'admittime', 'dischtime', 'intime', 'outtime', 'deathtime']
for thing in dates_and_times:
    print("converting {}".format(thing))
    new_series = pd.to_datetime(ptnt_demog2.loc[:,thing])
    ptnt_demog2.loc[:,thing] = new_series
    
    '''
    rows = ptnt_demog2.index
    for row in rows:
        ptnt_demog2.loc[:,thing] = pd.to_datetime(ptnt_demog2.loc[:,thing])
    '''
print("converted to date times")
display(ptnt_demog2.head())

In [ ]:
ptnt_demog2.dtypes

In [ ]:
ptnt_demog2.shape

## DROP OUTLIER AGE VALUES

In [ ]:
age_replace_vals = list(ptnt_demog2[ptnt_demog2['age'] > 110]['age'].unique())
display(age_replace_vals)

ptnt_demog2['age'].replace(age_replace_vals, np.nan, inplace = True)
ptnt_demog2['age'].dropna().describe()

## DISPLAY DESCRIPTIVE STATS

In [ ]:
ptnt_demog2.columns

In [ ]:
display(ptnt_demog2.icu_stay.dropna().describe())
display(ptnt_demog2.hosp_stay.dropna().describe())
display(ptnt_demog2[ptnt_demog2.icu_stay <= 0])
display(ptnt_demog2[ptnt_demog2.hosp_stay <=0])
icu_stay_low = list(ptnt_demog2[ptnt_demog2['icu_stay'] <=0]['icu_stay'].unique())
age_replace_vals = list(ptnt_demog2[ptnt_demog2['age'] > 110]['age'].unique())

#ptnt_demog2.loc[:,'subject_id'] = ptnt_demog2.index
#ptnt_demog2.index = ptnt_demog2.icustay_id
#ptnt_demog2.drop('icustay_id', axis = 1, inplace = True)
#ptnt_demog2.head()

Date and time data imported in string format is converted to pandas.datetime objects

## Extracting diagnoses information using code grabbed from benchmarking exercise

In [23]:
ptnt_demog_data2_df.head()

,hospital_expire_flag,age,icu_stay,hosp_stay,icustay_id,first_careunit,gender,marital_status,ethnicity,insurance,admission_type
0,0,76.0,146.0,259.0,211552,MICU,M,MARRIED,WHITE,Medicare,EMERGENCY
1,0,76.0,146.0,259.0,211552,MICU,M,MARRIED,WHITE,Medicare,EMERGENCY
2,0,76.0,146.0,259.0,211552,MICU,M,MARRIED,WHITE,Medicare,EMERGENCY
3,0,76.0,146.0,259.0,211552,MICU,M,MARRIED,WHITE,Medicare,EMERGENCY
4,0,76.0,146.0,259.0,211552,MICU,M,MARRIED,WHITE,Medicare,EMERGENCY


In [22]:
ptnt_demog_data2_df, diagnoses2 = ptnt_demog.create_diagnoses_df(ptnt_demog_data2_df, diagnoses_bm, diagnoses)

# #phenotypes = add_hcup_ccs_2015_groups(diagnoses, yaml.load(open(args.phenotype_definitions, 'r')))
# definitions = yaml.load(open('hcup_ccs_2015_definitions.yaml', 'r'))

# diagnoses = ptnt_demog[['hadm_id', 'icd9_code', 'short_title']].copy()

# # create mapping of hcup_ccs_2015_definitions to diagnoses icd9 codes
# def_map = {}
# for dx in definitions:
#     for code in definitions[dx]['codes']:
#         def_map[code] = (dx, definitions[dx]['use_in_benchmark'])

# print "map created"
# # map hcup_ccs_2015 definitions to icd9 diagnoses codes
# diagnoses['HCUP_CCS_2015'] = diagnoses.icd9_code.apply(lambda c: def_map[c][0] if c in def_map else None)
# diagnoses['USE_IN_BENCHMARK'] = diagnoses.icd9_code.apply(lambda c: int(def_map[c][1]) if c in def_map else None)
# #diagnoses['subject_id'] = diagnoses.index
# #diagnoses.set_index(np.arange(diagnoses.shape[0]), inplace = True)


# # create dataframe from the def_map dict so that we can isolate the 
# # definitions that are used in benchmarking

# def_map_df = pd.DataFrame.from_dict(def_map, orient = 'index')
# def_map_df.columns = ['Diagnoses', 'Benchmark']
# diagnoses_bm = list(def_map_df[def_map_df.Benchmark == True].drop_duplicates('Diagnoses').Diagnoses)
# icustays = list(ptnt_demog2.index)

# # create dataframe with hcup_ccp diagnoses benchmark categories as columns and
# # icustay_id information as indices. if the diagnosis is present for a given icustay the 
# # value is 1, otherwise 0. 

# diagnoses2 = pd.DataFrame(columns = diagnoses_bm, index = icustays)
# diagnoses2.fillna(0, inplace = True)
# print "created empty diagnosis dataframe"
# for row in diagnoses.iterrows():
#     if row[1]['USE_IN_BENCHMARK'] == 1:
#         diagnoses2.loc[row[0]][row[1]['HCUP_CCS_2015']] = 1

# print "filled diagnosis dataframe"
# diagnoses2.head()

In [ ]:
ptnt_demog_data2_df.drop(['subject_id', 'deathtime', 'hadm_id'], inplace = True, axis = 1)
cols = list(ptnt_demog_data2_df.columns)
cols.insert(0, cols.pop(cols.index('hospital_expire_flag')))
ptnt_demog_data2_df = ptnt_demog_data2_df[cols]


In [ ]:
diagnoses.dtypes

In [ ]:
diagnoses2.columns


## CONVERT CONTINUOUS TO CATEGORICAL DATA

In [ ]:
# NEED TO DO SOME OVERVIEW OF DATA INCLUDING DISTRIBUTIONS, SUMMARY STATISTICS ETC
# SINCE WE HAVE SO MUCH DATA, WE ARE JUMPING AHEAD..... IN TIME!! 
# AND SELECTING FEATURES THAT WERE USED IN THE FINAL CLASSIFICATION

demog_select_features_df = ptnt_demog2[['hospital_expire_flag', 'icu_stay', 'age', 'first_careunit', 'admission_type']]
demog_select_features_df.head()

demog_select_features_df[demog_select_features_df.columns[:3]].dropna().groupby('hospital_expire_flag').describe()


pd.DataFrame(demog_select_features_df.groupby('hospital_expire_flag').first_careunit.value_counts()).plot.bar(stacked = True)

icu_admits_df = pd.DataFrame(demog_select_features_df['first_careunit'].value_counts()/demog_select_features_df.shape[0])
icu_admits_df.index

icu_mortality_rates_df = pd.DataFrame(demog_select_features_df.groupby('first_careunit').hospital_expire_flag.sum()/ \
demog_select_features_df.groupby('first_careunit').hospital_expire_flag.count())
icu_admits_df.merge(icu_mortality_rates_df, left_index = True, right_index = True, 
                       how = 'left', sort = True)

demog_select_features_df[demog_select_features_df['first_careunit'] == 'MICU']['first_careunit'].count()



admission_types_df = pd.DataFrame(demog_select_features_df['admission_type'].value_counts()/demog_select_features_df.shape[0])

admission_type_mortality_rates_df = pd.DataFrame(demog_select_features_df.groupby('admission_type').hospital_expire_flag.sum()/ \
demog_select_features_df.groupby('admission_type').hospital_expire_flag.count())

admission_types_df.merge(admission_type_mortality_rates_df, left_index = True, right_index = True, 
                       how = 'left', sort = True)
'''
earlier versions included these factors but the  process has been iterative so we 
pre-selected relevant features above
diagnoses2['GCS Total_15', 'GCS Total_3', 'GCS Total_6', 'GCS Total_4', 
                          'Respiratory Failure','Shock', 'Septicemia (except in labor)', 'Acute and unspecified renal failure','Other liver diseases',
                         'Fluid and electrolyte disorders',
                          'Pneumonia (except that caused by tuberculosis or sexually transmitted disease)',
                            'Acute cerebrovascular disease']
'''

In [ ]:
def quant_cats(feature, Q1, Q2, Q3):
    if feature <=Q1:
        return 'Q0'
    elif (feature >Q1 and feature <= Q2):
        return 'Q1'
    elif (feature > Q2 and feature <= Q3):
        return 'Q2'
    elif feature > Q3:
        return 'Q3'
    

ptnt_demog3 = ptnt_demog2.copy()
demog_stats = ptnt_demog3[ptnt_demog3.columns[1:4]].dropna().describe()
for col in ptnt_demog3.columns[1:4]:
        Q1 = demog_stats[col].loc['25%']
        Q2 = demog_stats[col].loc['50%']
        Q3 = demog_stats[col].loc['75%']
        ptnt_demog3[col] = ptnt_demog3[col].apply(lambda x: quant_cats(x, Q1, Q2, Q3))



In [ ]:
demog_stats

## CONVERT CATEGORICAL TO DUMMY VARIABLES

In [ ]:
ptnt_demog2.dtypes


In [ ]:
ptnt_demog3.dtypes


In [ ]:
dummies = ptnt_demog2[ptnt_demog3.columns[:1]]

#display(monkey.head())
for col in ptnt_demog3.columns[1:]:
    chimp = pd.get_dummies(ptnt_demog3[col], prefix = col)
    dummies = dummies.merge(chimp, left_index = True, right_index = True, 
                       how = 'left', sort = True)

dummies.head()
    

## MERGE DUMMY VARIABLES AND DIAGNOSES

dummies = dummies.merge(diagnoses2, left_index = True, right_index = True, 
                       how = 'left')
dummies.dropna().shape

In [ ]:
diags2 = list(diagnoses2.columns)
#diags.pop(diags.index('Diabetes mellitus without complication'))
diags = [
 'hospital_expire_flag',
 'Acute cerebrovascular disease',
 'Other liver diseases',
 'Acute and unspecified renal failure',
 'Fluid and electrolyte disorders',
 'Septicemia (except in labor)',
 'Pneumonia (except that caused by tuberculosis or sexually transmitted disease)',
 'Respiratory failure; insufficiency; arrest (adult)',
 'Shock'
 ]


plt.subplots(figsize=(13,6))
diags_df = dummies[diags]
#diags_df.head()

diags_df[diags_df.columns[1:]][diags_df['hospital_expire_flag']==1].sum().plot.bar(
        alpha=0.5,label= 'Non_Survival')
diags_df[diags_df.columns[1:]][diags_df['hospital_expire_flag']==0].sum().plot.bar(
        alpha=0.1,label= 'Survival')
    

## SELECT FEATURES AND WRITE TO FILE. 

In [ ]:


root = os.getcwd() + '/features/'
name = 'Ptnt_Demog_Features.csv'
name2 = 'Ptnt_Demog_FeaturesScores.csv'
frame = dummies
X = frame[frame.columns[1:]]
y = frame['hospital_expire_flag']

        
# SELECT K BEST FEATURES BASED ON CHI2 SCORES
selector = SelectKBest(score_func = chi2, k = 'all')
selector.fit(X, y)
p_vals = pd.Series(selector.pvalues_, name = 'p_values', index = X.columns)
scores = pd.Series(selector.scores_, name = 'scores', index = X.columns)
features_df = pd.concat([p_vals, scores], axis = 1)
features_df.sort_values(by ='scores', ascending = False, inplace = True)
print "Feature scores/p_values in descending/ascending order"
display(features_df.head(20))

best_features = frame[features_df[features_df.p_values < .001].index]

frame = pd.DataFrame(y).merge(best_features, left_index = True, right_index = True, 
                how = 'left', sort = True)


print "{}     {}".format(name, frame.shape)


# CODE TO REORDER FEATURES ALPHABETICALLY. MAKES FOR CLEANER COLUMNS, GROUPED BY TYPE
# BUT WHAT WE MAY WANT IS THE ORDER OF FEATURE SIGNIFICANCE PRESERVCED. 
'''
cols = list(frame.columns)
cols.sort()
cols.insert(0, cols.pop(cols.index('hospital_expire_flag')))
frame = frame[cols]
'''

print "head of selected feature frame "
display(frame.head())
frame.to_csv(root + name)
features_df[features_df.p_values < .001].to_csv(root + name2)
y = pd.DataFrame(y)
y.to_csv(root + 'outcomes.csv')